<a href="https://colab.research.google.com/github/OlimpiaGarcia/Book_algorithm/blob/main/model_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
# imports generales
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import missingno  as msno
import ast
import xgboost as xgb

# imports de modelo
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler


# imports de metricas
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# imports de red neuronal
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

# conexión con google
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [48]:
ruta_dir_test = "/content/drive/MyDrive/Colab Notebooks/octavo/benji/reto /data/train"

In [49]:
data = pd.read_csv(os.path.join(ruta_dir_test, 'train_scaled.csv'), encoding='utf-8')

In [50]:
text = pd.read_csv(os.path.join(ruta_dir_test, 'train_text.csv'), encoding='utf-8')

# Hacer que ambos DF usen la columna ID como indice

In [51]:
data.set_index('id', inplace=True)
text.set_index('id', inplace=True)

# Implementar KNN

In [52]:
def recomendar_librosKNN(id_libro, df_numerico, df_text, n_recomendaciones=10):

  #  KNN
  knn = NearestNeighbors(
      metric='cosine',
      algorithm='auto')
  knn.fit(df_numerico)

  # Vector del libro base
  vector_libro = df_numerico.loc[[id_libro]]

  # Obtener vecinos (sumamos 1 porque el primer resultado será el libro mismo)
  distancias, indices = knn.kneighbors(vector_libro, n_neighbors=n_recomendaciones + 1)

  # Ignorar el primer resultado (es el propio libro)
  indices_recomendados = indices[0][1:]
  ids_recomendados = df_numerico.iloc[indices_recomendados].index

  # Combinar con texto y devolver
  recomendaciones = df_text.loc[ids_recomendados][['book_title', 'book_details']]
  return recomendaciones, ids_recomendados

## Correr KNN

In [81]:
# Solicitar cuantos libros parecidos quieres:
#recomendaciones = int(input("¿Cuántos libros parecidos quieres?"))

# el libro es harry Potter 5
recomendar_librosKNN(10611, data, text, n_recomendaciones=10)


,book_title,book_details
id,,
12399,Harry Potter and the Goblet of Fire,It is the summer holidays and soon Harry Potte...
12693,Harry Potter and the Deathly Hallows,"Harry has been burdened with a dark, dangerous..."
7595,Harry Potter and the Sorcerer's Stone,Harry Potter has no idea how famous he is. Tha...
9483,The Tales of Beedle the Bard,"""The Tales of Beedle the Bard"" contains five r..."
10061,Harry Potter and the Cursed Child. Parts One a...,Based on an original new story by J.K. Rowling...
7671,Harry Potter and the Cursed Child: Parts One a...,"The eighth story, nineteen years later...It wa..."
8666,The Harry Potter Collection 1-4,"The exciting tales of Harry Potter, the young ..."
11682,"Harry Potter Boxed Set, Books 1-5",Box Set containing Harry Potter and the Sorcer...
7145,Harry Potter Schoolbooks: Quidditch Through th...,SHS0545007984 Features: -Subjects: Literacy Ce...


# Target:
Saber  que tan buenos son los libros recomendados

### para esto necesitamos la columna "average_rating" sin escalar

In [54]:
rating = pd.read_csv(os.path.join(ruta_dir_test, 'train_data.csv'), encoding='utf-8')

In [55]:
rating['average_rating']

,average_rating
0,4.24
1,4.26
2,4.00
3,4.03
4,3.84
...,...
12975,3.32
12976,3.58
12977,4.21
12978,4.46


In [56]:
data['average_rating_original'] = rating['average_rating'].values
data.drop('average_rating', axis=1, inplace=True)

In [57]:
data.sample(5).T

id,50,1416,4731,10768,6117
num_pages,0.098893,-0.320438,0.632946,0.830743,-0.411424
num_ratings,-0.229463,-0.249278,0.026749,-0.246061,-0.244674
num_reviews,-0.267369,-0.297590,0.756713,-0.303153,-0.258139
rating_5,-0.192548,-0.222140,0.030243,-0.194245,-0.202204
rating_4,-0.250017,-0.264421,0.031346,-0.280603,-0.267461
...,...,...,...,...,...
يحيى بن شرف النووي,-0.008778,-0.008778,-0.008778,-0.008778,-0.008778
يوسف زيدان,0.000000,0.000000,0.000000,0.000000,0.000000
ปรีดี พนมยงค์,0.000000,0.000000,0.000000,0.000000,0.000000
နုနုရည်အင်းဝ,-0.008778,-0.008778,-0.008778,-0.008778,-0.008778


## Creamos la clase objetivo a predecir: Rating_class

Asignamos los valores de los ratings a una descripción y creamos la clase:

| Rango rating | Clase | Descripción |
| ------------ | ----- | ----------- |
| <= 2.5       | 0     | Malo        |
| >2.5 y <=2.9 | 1     | Regular     |
| >3 y <=4.0   | 2     | Bueno       |
| >4.0         | 3     | Excelente   |


Creamos la columna rating class y la llenamos con las clases de las distintas categorias usnado cut para seleccionar los intrvalos, bins son los rangos y labels son las clases que van a quedar en la columna

In [58]:
data['rating_class'] = pd.cut(
  data['average_rating_original'],
  bins=[-float('inf'), 2.5, 3.5, 4.0, float('inf')],
  labels=[0, 1, 2, 3]
)

In [59]:
# convertir la columna de category a int por que los modelos solo trabajan con numeros
data['rating_class'] = data['rating_class'].astype(int)

Checar como queedaron las clases:

In [60]:
print("Distribución de clases en rating_class:")
print(data['rating_class'].value_counts().sort_index())

Distribución de clases en rating_class:
rating_class
0      33
1     669
2    5348
3    6930
Name: count, dtype: int64


# cargamos el modelo

In [61]:
# cargar el modelo nuevamente de ser necesario:
modelo_cargado = load_model("mejor_modelo1.keras")
modelo_cargado2 = load_model("mejor_modelo2.keras")

# nueva data

In [82]:
new_data = pd.read_csv("/content/data_benji_test - Hoja 1.csv", index_col=False, header=None, encoding="utf-8")
new_text = pd.read_csv("/content/text_benji_test - Hoja 1.csv", index_col=False, header=None,encoding="utf-8" )

In [83]:
print(new_data.shape)
print(new_text.shape)

print(data.shape)
print(text.shape)

(9, 8353)
(9, 3)
(12980, 8353)
(12980, 3)


# Anadimos nueva data con la data vieja

In [84]:
new_data.columns = data.columns
df_actualizado = pd.concat([data, new_data], ignore_index=True)

In [85]:
new_text.columns = text.columns
df_actualizado_text = pd.concat([text, new_text], ignore_index=True)

In [86]:
df_actualizado

,num_pages,num_ratings,num_reviews,rating_5,rating_4,rating_3,rating_2,rating_1,12th Century,13th Century,...,مصطفى صادق الرافعي,مصطفى محمود,نور عبدالمجيد,هبة سيد عبد العاطي,يحيى بن شرف النووي,يوسف زيدان,ปรีดี พนมยงค์,နုနုရည်အင်းဝ,average_rating_original,rating_class
0,-0.842623,-0.217577,-0.286589,-0.177570,-0.242676,-0.250705,-0.223988,-0.159908,-0.012414,-0.012414,...,-0.008778,-0.015205,-0.008778,-0.008778,-0.008778,0.0,0.0,-0.008778,4.24,3
1,-0.399557,-0.249876,-0.317189,-0.211447,-0.269901,-0.284236,-0.250421,-0.178224,-0.012414,-0.012414,...,-0.008778,-0.015205,-0.008778,-0.008778,-0.008778,0.0,0.0,-0.008778,4.26,3
2,-0.000006,-0.239183,-0.304165,-0.207334,-0.258941,-0.260049,-0.225720,-0.166120,-0.012414,-0.012414,...,-0.008778,-0.015205,-0.008778,-0.008778,-0.008778,0.0,0.0,-0.008778,4.00,2
3,-0.411424,-0.268686,-0.323764,-0.229737,-0.290560,-0.301126,-0.260874,-0.184972,-0.012414,-0.012414,...,-0.008778,-0.015205,-0.008778,-0.008778,-0.008778,0.0,0.0,-0.008778,4.03,3
4,0.162188,-0.227681,-0.261426,-0.208593,-0.236048,-0.231541,-0.211471,-0.165478,-0.012414,-0.012414,...,-0.008778,-0.015205,-0.008778,-0.008778,-0.008778,0.0,0.0,-0.008778,3.84,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12984,-0.320438,-0.266131,-0.325661,0.895113,-0.227264,-0.286931,-0.300526,-0.259343,-0.183794,-0.012414,...,-0.008778,-0.015204,-0.008778,-0.008778,-0.008778,0.0,0.0,-0.008778,3.10,2
12985,-0.458896,-0.240506,-0.276031,-0.287741,-0.211930,-0.256107,-0.260194,-0.222990,-0.166228,-0.012414,...,-0.008778,-0.015204,-0.008778,-0.008778,-0.008778,0.0,0.0,-0.008778,4.10,3
12986,0.280866,-0.268581,-0.323448,1.137750,-0.229451,-0.290612,-0.301146,-0.261207,-0.185079,-0.012414,...,-0.008778,-0.015204,-0.008778,-0.008778,-0.008778,0.0,0.0,-0.008778,3.20,2
12987,-0.403512,3.071097,3.860633,0.045885,2.079362,4.020799,4.005070,2.808982,1.435286,-0.012414,...,-0.008778,-0.015204,-0.008778,-0.008778,-0.008778,0.0,0.0,-0.008778,4.50,3


# separamos X

Creamos una x donde dropeamos el average rating y el rating class ya que ese lo vamos a usar como la columna a predecir

In [87]:
# Separar X (features) e y (target)
X = data.drop(columns=['average_rating_original', 'rating_class'])


Creamos una x alternativa donde dropeamos tambien las columnas de rating para ver como afectan al modelo

In [88]:
X1 = data.drop(columns= ['average_rating_original', 'rating_class','rating_5', 'rating_4', 'rating_3', 'rating_2', 'rating_1'])


# predecimos

In [89]:
# predecir modelo 1
y_pred_nn1 = modelo_cargado.predict(X)

406/406 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [90]:
y_pred_nn1 = pd.DataFrame(y_pred_nn1, columns=['y_pred_knn1'])
y_pred_nn1

,y_pred_knn1
0,2.886607
1,1.213431
2,1.884948
3,2.982721
4,2.985212
...,...
12975,1.119220
12976,2.003450
12977,1.865014
12978,2.677329


In [91]:
# predecir modelo 2
y_pred2_nn2 = modelo_cargado2.predict(X1)

406/406 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [92]:
y_pred2_nn2 = pd.DataFrame(y_pred2_nn2, columns=['y_pred_knn2'])
y_pred2_nn2

,y_pred_knn2
0,2.961157
1,2.024027
2,1.993223
3,3.006481
4,3.031995
...,...
12975,0.954086
12976,1.987437
12977,1.748921
12978,2.921277


# juntarlos